# Building a Movie Recommendation Agent with LangChain and Together AI

This notebook demonstrates how to create a movie recommendation agent that:

1. **Remembers your preferences** across sessions using LangChain memory
2. **Uses external APIs** (OMDB for movie data, JustWatch for streaming availability)
3. **Makes intelligent recommendations** based on your movie taste
4. **Compares reasoning approaches** using ReAct vs. simpler prompting strategies

The agent uses Together AI's Llama 3.1 8B model for powerful, cost-effective recommendations.

In [ ]:
# Install necessary packages
!pip install langchain langchain-openai langchain-together requests justwatch langsmith

## Import Libraries

Let's import all the necessary libraries for our movie recommendation agent.

In [ ]:
import os
import requests
import json
from typing import Dict, Any, List

# LangChain imports
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether  # Added import for Together AI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Import JustWatch for streaming availability checks
from justwatch import JustWatch

# Import LangSmith for tracing
from langsmith import traceable

# Enable LangSmith tracing (optional)
os.environ["LANGSMITH_TRACE"] = "True"  # Enable tracing
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"  # Set the LangSmith endpoint
os.environ["LANGSMITH_API_KEY"] = ""  # Enter Your LangSmith API Key
os.environ["LANGSMITH_PROJECT"] = "chapter-10-3"  # Set the LangSmith project


## API Keys Setup

You'll need API keys for Together AI and OMDB. The OMDB API key is free and can be obtained from http://www.omdbapi.com/apikey.aspx.

In [ ]:
# Set your API keys here
os.environ["TOGETHER_API_KEY"] = ""  # Replace with your Together AI key
os.environ["OMDB_API_KEY"] = ""          # Replace with your OMDB API key

# Verify API keys are set
if not os.environ.get("TOGETHER_API_KEY"):
    raise ValueError("Please set your TOGETHER_API_KEY environment variable")
if not os.environ.get("OMDB_API_KEY"):
    raise ValueError("Please set your OMDB_API_KEY environment variable")

## API Helper Functions

These functions will communicate with movie databases and streaming services.

In [12]:
def get_movie_details(movie_title: str) -> Dict[str, Any]:
    """Get detailed information about a movie from OMDB API."""
    api_key = os.environ["OMDB_API_KEY"]
    url = f"http://www.omdbapi.com/?apikey={api_key}&t={movie_title}&r=json"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("Response") == "True":
            return data
    return None

def check_availability(movie_title: str, country: str = "CA") -> Dict[str, Any]:
    """Check if a movie is available on streaming services in the specified country."""
    try:
        just_watch = JustWatch(country=country)
        results = just_watch.search_for_item(query=movie_title)
        
        if results and 'items' in results and results['items']:
            movie = results['items'][0]
            providers = []
            
            # Extract provider information if available
            if 'offers' in movie:
                providers = [offer.get('provider_name', 'Unknown') 
                             for offer in movie['offers'] 
                             if 'provider_name' in offer]
            
            return {
                "title": movie.get("title"),
                "available": True,
                "providers": providers
            }
        return {"available": False}
    except Exception as e:
        print(f"Error checking availability: {e}")
        return {"available": False, "error": str(e)}

## Memory Setup

We'll create a simple file-based memory system to store user preferences across sessions.

In [13]:
class PreferencesStore:
    """Simple memory storage for user movie preferences."""
    
    def __init__(self, file_path="user_preferences.json"):
        self.file_path = file_path
        self.preferences = self._load()
    
    def _load(self):
        """Load preferences from file or create empty preferences."""
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as f:
                return json.load(f)
        return {}
    
    def _save(self):
        """Save preferences to file."""
        with open(self.file_path, 'w') as f:
            json.dump(self.preferences, f, indent=2)
    
    def get(self, user_id="default_user"):
        """Get user preferences or initialize if not present."""
        if user_id not in self.preferences:
            self.preferences[user_id] = {
                "liked_movies": [],
                "genres": [],
                "country": "CA"  # Default to Canada
            }
            self._save()
        return self.preferences[user_id]
    
    def update_movies(self, movies, user_id="default_user"):
        """Add movies to user's liked list."""
        prefs = self.get(user_id)
        
        # Handle input formats: list or comma-separated string
        if isinstance(movies, str):
            movies = [movie.strip() for movie in movies.split(",")]
        
        for movie in movies:
            if movie and movie not in prefs["liked_movies"]:
                prefs["liked_movies"].append(movie)
        
        self._save()
        return prefs["liked_movies"]
    
    def update_genres(self, genres, user_id="default_user"):
        """Update user's favorite genres."""
        prefs = self.get(user_id)
        
        if isinstance(genres, str):
            genres = [genre.strip() for genre in genres.split(",")]
        
        for genre in genres:
            if genre and genre not in prefs["genres"]:
                prefs["genres"].append(genre)
        
        self._save()
        return prefs["genres"]
    
    def update_country(self, country, user_id="default_user"):
        """Update user's country for availability checks."""
        prefs = self.get(user_id)
        prefs["country"] = country
        self._save()
        return country

# Initialize preferences store
preferences = PreferencesStore()

## Tool Functions

Define functions that will become tools for our LangChain agent.

In [14]:
def get_user_preferences(user_id="default_user"):
    """Get the current user preferences as a formatted string."""
    prefs = preferences.get(user_id)
    return f"Favorite Movies: {', '.join(prefs['liked_movies'])}\nFavorite Genres: {', '.join(prefs['genres'])}\nCountry: {prefs['country']}"

def update_liked_movies(movies_input, user_id="default_user"):
    """Update the user's list of liked movies."""
    updated = preferences.update_movies(movies_input, user_id)
    return f"Updated favorite movies: {', '.join(updated)}"

def update_genres(genres_input, user_id="default_user"):
    """Update the user's favorite genres."""
    updated = preferences.update_genres(genres_input, user_id)
    return f"Updated favorite genres: {', '.join(updated)}"

def update_country(country, user_id="default_user"):
    """Update the user's country for availability checks."""
    updated = preferences.update_country(country, user_id)
    return f"Updated country to: {updated}"

def get_movie_info(movie_title):
    """Get detailed information about a movie."""
    movie_data = get_movie_details(movie_title)
    if movie_data:
        return json.dumps({
            "Title": movie_data.get("Title"),
            "Year": movie_data.get("Year"),
            "Director": movie_data.get("Director"),
            "Plot": movie_data.get("Plot"),
            "Ratings": movie_data.get("Ratings", []),
            "Genre": movie_data.get("Genre")
        }, indent=2)
    return f"Could not find information for '{movie_title}'"

def check_movie_availability(movie_title, user_id="default_user"):
    """Check if a movie is available on streaming platforms in the user's country."""
    country = preferences.get(user_id)["country"]
    availability = check_availability(movie_title, country)
    
    if availability.get("available", False):
        providers = ", ".join(availability.get("providers", ["Unknown"]))
        return f"'{movie_title}' is available in {country} on: {providers}"
    return f"'{movie_title}' does not appear to be available for streaming in {country}"

def generate_recommendations(user_id="default_user"):
    """Generate movie recommendations based on user preferences."""
    prefs = preferences.get(user_id)
    liked_movies = prefs["liked_movies"]
    genres = prefs["genres"]
    country = prefs["country"]
    
    # Check if we have enough preference data
    if not liked_movies and not genres:
        return "Please add some favorite movies or genres first so I can make recommendations."
    
    # Use Together AI model instead of OpenAI
    llm = ChatTogether(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0.7
    )
    prompt = PromptTemplate(
        input_variables=["movies", "genres", "country"],
        template="""Based on the following user preferences, recommend 5 movies:
        
        Liked Movies: {movies}
        Preferred Genres: {genres}
        Country: {country}
        
        Provide only the movie titles as a comma-separated list.
        """
    )
    
    chain = LLMChain(llm=llm, prompt=prompt)
    recommendations = chain.run(
        movies=', '.join(liked_movies), 
        genres=', '.join(genres), 
        country=country
    )
    
    # Parse recommendations and check availability
    movie_titles = [title.strip() for title in recommendations.split(',')]
    results = []
    
    for title in movie_titles:
        # Check availability
        avail = check_availability(title, country)
        # Get movie details
        details = get_movie_details(title)
        
        if details:
            results.append({
                "title": title,
                "year": details.get("Year", "Unknown"),
                "genre": details.get("Genre", "Unknown"),
                "available": avail.get("available", False),
                "providers": avail.get("providers", [])
            })
    
    # Format the results
    output = "Here are your personalized recommendations:\n\n"
    
    for movie in results:
        output += f"🎬 {movie['title']} ({movie['year']}) - {movie['genre']}\n"
        if movie['available']:
            output += f"   ✅ Available on: {', '.join(movie['providers'])}\n"
        else:
            output += "   ❌ Not available for streaming in your country\n"
    
    return output

## Create LangChain Tools

Now we'll create the tools for our agent using the functions we defined above.

In [15]:
tools = [
    Tool(
        name="get_user_preferences",
        func=get_user_preferences,
        description="Get the current user preferences including favorite movies, genres, and country"
    ),
    Tool(
        name="update_liked_movies",
        func=update_liked_movies,
        description="Update the user's favorite movies. Input should be a comma-separated list of movie titles."
    ),
    Tool(
        name="update_genres",
        func=update_genres,
        description="Update the user's favorite genres. Input should be a comma-separated list of genres."
    ),
    Tool(
        name="update_country",
        func=update_country,
        description="Update the user's country for availability checks. Input should be a country code like 'CA' for Canada or 'US' for United States."
    ),
    Tool(
        name="get_movie_info",
        func=get_movie_info,
        description="Get detailed information about a specific movie. Input should be the movie title."
    ),
    Tool(
        name="check_movie_availability",
        func=check_movie_availability,
        description="Check if a specific movie is available on streaming platforms in the user's country. Input should be the movie title."
    ),
    Tool(
        name="generate_recommendations",
        func=generate_recommendations,
        description="Generate personalized movie recommendations based on the user's preferences."
    )
]

## Creating Different Reasoning Approaches

We'll create two different agent types to compare:
1. ReAct Agent - Uses structured reasoning and explicit step-by-step thinking
2. Simple Agent - Uses minimal prompting without explicit reasoning instructions

In [33]:
def create_react_agent():
    """Create a ReAct agent that uses explicit reasoning steps."""
    
    # Define a system message that guides the agent to use ReAct reasoning
    react_system_message = """You are MovieMaster, an intelligent movie recommendation assistant that uses careful step-by-step reasoning.

    When responding to user requests, follow these steps:
    1. ANALYZE: First, analyze what information you have and what you need
    2. PLAN: Decide which tools to use in what order
    3. EXECUTE: Use the selected tools and examine their outputs
    4. REFLECT: Evaluate if the results are helpful or if you need more information
    5. RESPOND: Provide a clear, helpful response to the user

    Your capabilities:
    - Remember user preferences (favorite movies, genres, country)
    - Look up movie information and check streaming availability
    - Generate personalized movie recommendations

    Always break down your reasoning process explicitly when deciding which tools to use.
    """
    
    # Initialize the language model with Together AI
    llm = ChatTogether(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", 
        temperature=0,
        max_tokens=1024
    )
    
    # Create conversation memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # Initialize the agent with ReAct instructions - use a different agent type
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # Changed agent type
        verbose=True,
        memory=memory,
        system_message=react_system_message
    )
    
    return agent

def create_simple_agent():
    """Create a simple agent with minimal reasoning instructions."""
    
    # Define a simple system message with minimal reasoning guidance
    simple_system_message = """You are MovieMaster, a helpful movie recommendation assistant.

    You can help users find movies they'll enjoy by using these tools:
    - Remember user preferences (movies, genres, country)
    - Look up movie information and check availability
    - Generate personalized movie recommendations

    Be conversational and provide helpful suggestions.
    """
    
    # Initialize the language model with Together AI
    llm = ChatTogether(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", 
        temperature=0,
        max_tokens=1024
    )
    
    # Create conversation memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    # Initialize the agent with simple instructions - use a different agent type
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # Changed agent type
        verbose=True,
        memory=memory,
        system_message=simple_system_message
    )
    
    return agent

In [34]:
react_agent = create_react_agent()
simple_agent = create_simple_agent()

@traceable
def run_react_agent(query: str):
    """Run the ReAct agent with the given query and return the result."""
    result = react_agent.invoke({"input": query})
    # Debug print to help identify what's in the result
    print("DEBUG - Result keys:", result.keys())
    return result["output"]

@traceable
def run_simple_agent(query: str):
    """Run the simple agent with the given query and return the result."""
    result = simple_agent.invoke({"input": query})
    return result["output"]

## Testing and Comparing Agent Approaches

Let's test both agent types with the same queries to compare their performance.

### Test 1: Setting Initial Preferences

In [ ]:
test_query = "I like movies like The Shawshank Redemption, Inception, and The Dark Knight. I enjoy drama, sci-fi, and thriller genres. I'm in Canada."

print("=== ReAct Agent Response ===")
react_response = run_react_agent(test_query)
print(react_response)

print("\n=== Simple Agent Response ===")
simple_response = run_simple_agent(test_query)
print(simple_response)

### Test 2: Getting Movie Recommendations

In [ ]:
test_query = "What movies would you recommend for me?"

print("=== ReAct Agent Response ===")
react_response = run_react_agent(test_query)
print(react_response)

print("\n=== Simple Agent Response ===")
simple_response = run_simple_agent(test_query)
print(simple_response)

### Test 3: Complex Multi-step Query

In [ ]:
test_query = "Can you tell me about the movie Interstellar and check if it's available in my country? Also, suggest similar movies that I might enjoy."

print("=== ReAct Agent Response ===")
react_response = run_react_agent(test_query)
print(react_response)

print("\n=== Simple Agent Response ===")
simple_response = run_simple_agent(test_query)
print(simple_response)

## Analysis: ReAct vs. Simple Prompting for Recommendation Agents

After testing both approaches, we can analyze the differences between the ReAct agent (with explicit reasoning steps) and the simple agent (with minimal prompting):

### ReAct Advantages:

1. **More methodical tool selection**: The ReAct agent tends to be more explicit about which tool it's using and why
2. **Better handling of complex queries**: For multi-step tasks, the structured reasoning helps ensure all parts are addressed
3. **More transparent process**: Users can better understand how the agent arrived at its recommendations

### Simple Agent Advantages:

1. **Faster responses**: With less overhead on reasoning steps, responses can be generated more quickly
2. **More conversational tone**: Less technical explanations can make the interaction feel more natural
3. **May be sufficient for simpler queries**: For straightforward requests, the extra reasoning may be unnecessary

### When to Use Each Approach:

- **Use ReAct** when recommendation tasks involve complex reasoning, multiple steps, or critical decisions where transparency matters
- **Use Simple Prompting** for more straightforward recommendation tasks where speed and conversational flow are priorities

For this specific movie recommendation use case, the ideal approach likely depends on the complexity of the query and how important transparency is to the user.